# Domain Decomposition

## Overview

### Questions

* What is a MPI rank? 
* How should I structure my scripts? 
* How does HOOMD-blue divide the simulation among the ranks? 
* What limitations prevent parallel execution?

### Objectives

* Demonstrate how **MPI** ranks are **processes** that communicate with the **Communicator**.
* Emphasize that scripts are a **single program** that can execute in serial or parallel.
* Warn about common pitfalls and show the correct way to avoid **deadlock**.
* Explain how HOOMD-blue divides the simulation State with a **domain decomposition** and how operations execute only on the local particles.
* Demonstrate the minimum **domain** size.
* Discuss how particles are placed in **domains** and how this can lead to uneven **load balancing**.

## Ranks and processes

When you run `mpirun -n 4 python3 script.py`, there will be 4 separate instances of `python` all executing `script.py` at the same time.
A script that prints a message will result in repeated output.
For example:

In [1]:
%pycat hello_world.py

print('Hello, world')


In [2]:
!mpirun -n 4 python3 hello_world.py

Hello, world
Hello, world
Hello, world
Hello, world


In general, **MPI** launches **n** separate **processes** with the given command.
These may or may not be on the same node in a **HPC** cluster, depending on how you request resources in your job script.
Each **process** launched this way is called a **rank** and is given a rank index.
In HOOMD-blue, the **Communicator** class (created by default with **Device**) gives you access to the **rank** information.

In [3]:
%pycat hello_hoomd.py

import hoomd
import os

device = hoomd.device.CPU()
rank = device.communicator.rank
pid = os.getpid()
print(f'Hello HOOMD-blue rank {rank} from process id {pid}')


In [4]:
!mpirun -n 4 python3 hello_hoomd.py

Hello HOOMD-blue rank 2 from process id 8842
Hello HOOMD-blue rank 3 from process id 8843
Hello HOOMD-blue rank 0 from process id 8840
Hello HOOMD-blue rank 1 from process id 8841


## Single program

HOOMD-blue scripts must be written as a **single program** executed in parallel.
In other words, all **ranks** must load the *same* input file, define the *same* operations with the *same* parameters and triggers, and run the *same* number of time steps.
HOOMD-blue requires this because the operations need to communicate between the ranks.
If you violate this in your script, it is likely at least one rank will **deadlock** while it waits for a message to be sent from another rank that will not be sent.
A **deadlock** means that the execution continues in loop waiting for a condition that will never be true.

While this is true for all HOOMD-blue operations, properties, and methods you call - it may not be the case for other libraries or methods in your script.
For example, we've seen several times now that calling `print` results in duplicated output.
The same applies when you use `open()` to open and write to a file.
In cases like these, execute your code only on rank 0 with a `if device.communicator.rank == 0:` check so that it only runs once.
For example, here is a modification to the previous `lj_performance.py` file that only prints the performance once:

In [5]:
%pycat lj_performance2.py

import hoomd

device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device)
sim.create_state_from_gsd(filename='random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.run(1000)

tps = sim.tps
if device.communicator.rank == 0:
    print(tps)


In [6]:
!mpirun -n 4 python3 lj_performance2.py

notice(2): Using domain decomposition: n_x = 1 n_y = 2 n_z = 2.
2496.398944522526


Note the pattern used here:
```python
tps = sim.tps
if device.communicator.rank == 0:
    print(tps)
```
The property `sim.tps` is accessed on all ranks, but printed only on rank 0.

If the code was instead like that below the property `sim.tps` would be accessed only on rank 0 and would **deadlock** the execution.:
```python
if device.communicator.rank == 0:
    print(sim.tps)
```

So, be careful using `if device.communicator.rank == 0:`.
The rich Python API appears simple, but any property access or method call on a HOOMD-blue object may result in a MPI communication that will deadlock when inside this condition.

<div class="alert alert-info">
    Scripts using <code>if device.communicator.rank == 0:</code> are compatible with serial execution where <code>rank</code> is always 0.
</div>

## Domain decomposition

When you create the State object in an MPI simulation on more than 1 rank, HOOMD-blue splits the simulation box into *k* x *l* x *m* **domains**.
The product of *k*, *l* and *m* is equal to the number of **ranks** you execute, so chose **n** to be values that factorize nicely given the constraints of your **HPC** system.
The domains are defined by planes that split the box.
By default, the planes are evenly spaced and chosen to minimize the surface area between the **domains**.

In [10]:
%pycat domain_decomposition.py

import hoomd

device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device)
sim.create_state_from_gsd(filename='random.gsd')

domain_decomposition = sim.state.domain_decomposition
if device.communicator.rank == 0:
    print(domain_decomposition)

split_fractions = sim.state.domain_decomposition_split_fractions
if device.communicator.rank == 0:
    print(split_fractions)

with sim.state.cpu_local_snapshot as snap:
    N = len(snap.particles.position)
    print(f'{N} particles on rank {device.communicator.rank}')


In [11]:
!mpirun -n 4 python3 domain_decomposition.py

notice(2): Using domain decomposition: n_x = 1 n_y = 2 n_z = 2.
(1, 2, 2)
([], [0.5], [0.5])
512 particles on rank 1
512 particles on rank 2
512 particles on rank 3
512 particles on rank 0


For example, this script run on 4 ranks chooses a 1 x 2 x 2 decomposition with the split planes in the center of the box.
`domain_decomposition_split_fractions` reports relative values between 0 and 1, so in this case a hypothetical 10 x 10 x 10 box would have split planes at y=5 and z=5 creating 4 **domains**.

Each **rank** is assigned one of these **domains** and stores the particles located inside it.
The operations execute on the particles local to each **rank**.
When the density of the system is uniform, each rank has approximately the same number of particles (as in the example above) and an even **load balancing**.
This is what allows the parallel simulations to run with faster performance: the same operation is being run on fewer particles so it takes less time.

However, when the density of the system is not uniform the default split planes lead to an uneven **load balancing** with a much greater number of particles on one rank compared to the others.
The performance of the overall simulation is limited by that of the slowest **rank**.
In the extreme case, imagine all the particles in the lower left of a very large box.
In this 1 x 2 x 2 domain decomposition, all particles would be on one **rank** and the simulation would take just as much time to execute as one **rank** alone.

Some computations, such as pair forces in MD or hard particle overlap checks in HPMC, need to interact with particles from a neighboring **domain**.
This establishes a lower limit on the **domain** size.
Given an interaction range *r_iteraction* (for MD, this is the sum of the largest pair potential *r_cut* and the neighbor list *buffer*), each x,y,z dimension of the **domain** must be larger than 2 * *r_interaction*.
HOOMD-blue raises an exception when this is violated.
For example, here is the Lennard-Jones script run on the `random.gsd` file before replicating to a larger size:

In [13]:
%pycat lj_domain_error.py

import hoomd

device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device)
sim.create_state_from_gsd(
    filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.run(0)


In [14]:
!mpirun -n 2 python3 lj_domain_error.py

notice(2): Using domain decomposition: n_x = 1 n_y = 1 n_z = 2.
Traceback (most recent call last):
  File "04-Parallel-Simulations-With-MPI/lj_domain_error.py", line 17, in <module>
Traceback (most recent call last):
  File "04-Parallel-Simulations-With-MPI/lj_domain_error.py", line 17, in <module>
    sim.run(0)
  File "/Users/joaander/build/hoomd/hoomd/simulation.py", line 435, in run
    sim.run(0)
  File "/Users/joaander/build/hoomd/hoomd/simulation.py", line 435, in run
    self._cpp_sys.run(steps_int, write_at_start)
RuntimeError: Communication error - 
Simulation box too small for domain decomposition.
r_ghost_max: 2.9
d.z/2: 2.275

    self._cpp_sys.run(steps_int, write_at_start)
RuntimeError: Communication error - 
Simulation box too small for domain decomposition.
r_ghost_max: 2.9
d.z/2: 2.275

--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 1 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT ca

In this section, you have seen how MPI ranks run as independent processes, learned that HOOMD-blue scripts need to execute all operations identically on all ranks, learned how to print output only once in MPI simulations, have seen how HOOMD splits particles across domains.
The next section of this tutorial shows you how to access particle data snapshots in MPI simulations.